# AI-Enhanced Qualitative Data Analysis

This notebook processes Excel workbooks containing qualitative data and uses AI models (BERTopic or Top2Vec) to automatically extract themes and topics for qualitative data analysis (QDA).

## Features
- Load multiple Excel sheets/tabs
- Extract qualitative text from specified columns
- Apply BERTopic for hierarchical theme extraction
- Interactive visualizations
- Export results for QDA tools (Taguette, QualCoder, etc.)

## Setup
1. Place your Excel workbook in the `data/raw/` directory
2. Update the `excel_file` path below
3. Customize column names if needed

In [ ]:
# Install required libraries if not already installed
# Uncomment the line below if running in an environment without these packages
# !uv install -r ../requirements.txt

In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import warnings
import glob

warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


In [2]:
# Step 1: Load the Excel workbook
# Replace 'your_workbook.xlsx' with the path to your Excel file
# Find the first .xlsx file in the data/raw directory
import time
excel_files = glob.glob('../data/raw/*.xlsx')

if excel_files:
    excel_file = excel_files[0]  # Use the first Excel file found
    print(f"Found Excel file: {excel_file}")
    
    # Load all sheets into a dictionary of DataFrames
    try:
        print("Loading workbook (this may take a moment for large files)...")
        start_time = time.time()
        sheets_dict = pd.read_excel(excel_file, sheet_name=None)
        load_time = time.time() - start_time
        
        print(f"✓ Successfully loaded workbook in {load_time:.1f} seconds")
        print(f"Loaded {len(sheets_dict)} sheets: {list(sheets_dict.keys())}")
        
        # Display basic info about each sheet
        total_rows = 0
        for sheet_name, df in sheets_dict.items():
            rows = df.shape[0]
            cols = df.shape[1]
            total_rows += rows
            print(f"  - {sheet_name}: {rows:,} rows, {cols} columns")
        
        print(f"\nCOMPLETE - Total rows across all sheets: {total_rows:,}")
            
    except FileNotFoundError:
        print(f"✗ Error: File '{excel_file}' not found.")
        print("Please place your Excel file in the data/raw/ directory and update the path above.")
        sheets_dict = {}
    except Exception as e:
        print(f"✗ Error loading Excel file: {e}")
        sheets_dict = {}
else:
    excel_file = None
    sheets_dict = {}
    print("✗ No Excel files found in ../data/raw/")
    print("Please place an Excel workbook (.xlsx) in the data/raw/ directory")

Found Excel file: ../data/raw/ALFD Call Center 10-1-2021 to 12-10-2025 - RAW.xlsx
Loading workbook (this may take a moment for large files)...
✓ Successfully loaded workbook in 1.9 seconds
Loaded 3 sheets: ['COLA FONL Unique', 'General Inqury Unique', 'Contact Reason Details']
  - COLA FONL Unique: 2,517 rows, 11 columns
  - General Inqury Unique: 1,583 rows, 11 columns
  - Contact Reason Details: 31,879 rows, 11 columns

COMPLETE - Total rows across all sheets: 35,979


In [3]:
# Step 2: Extract qualitative text data
# Configure text column names to extract from
text_columns = ['Contact Reason', 'Contact Reason Detail', 'Comment']  # Add your column names here

all_texts = []
text_sources = []  # Track which sheet each text came from

if sheets_dict:
    for sheet_name, df in sheets_dict.items():
        print(f"\nProcessing sheet: {sheet_name}")
        
        # Find text columns in this sheet
        available_text_cols = [col for col in text_columns if col in df.columns]
        
        if available_text_cols:
            print(f"  Found text columns: {available_text_cols}")
            
            # Extract text from each available column
            for col in available_text_cols:
                texts = df[col].dropna().astype(str).tolist()
                all_texts.extend(texts)
                text_sources.extend([f"{sheet_name}:{col}"] * len(texts))
                print(f"    - {col}: {len(texts)} texts extracted")
        else:
            print(f"  Warning: No text columns found in '{sheet_name}'. Available columns: {list(df.columns)}")
    
    print(f"\n✓ Total texts extracted: {len(all_texts)}")
    
    if all_texts:
        print(f"Sample text: {all_texts[0][:100]}...")
        
        # Save processed data
        processed_df = pd.DataFrame({
            'text': all_texts,
            'source': text_sources
        })
        processed_df.to_csv('../data/processed/extracted_texts.csv', index=False)
        print("✓ Processed texts saved to data/processed/extracted_texts.csv")
    else:
        print("✗ No texts were extracted. Please check your column names.")
else:
    print("No workbook loaded. Please check Step 1.")


Processing sheet: COLA FONL Unique
  Found text columns: ['Contact Reason', 'Contact Reason Detail', 'Comment']
    - Contact Reason: 2517 texts extracted
    - Contact Reason Detail: 2517 texts extracted
    - Comment: 2517 texts extracted

Processing sheet: General Inqury Unique
  Found text columns: ['Contact Reason', 'Contact Reason Detail', 'Comment']
    - Contact Reason: 1583 texts extracted
    - Contact Reason Detail: 1583 texts extracted
    - Comment: 1582 texts extracted

Processing sheet: Contact Reason Details
  Found text columns: ['Contact Reason', 'Contact Reason Detail', 'Comment']
    - Contact Reason: 31879 texts extracted
    - Contact Reason Detail: 31879 texts extracted
    - Comment: 31865 texts extracted

✓ Total texts extracted: 107922
Sample text: COLA/FONL Issues...
✓ Processed texts saved to data/processed/extracted_texts.csv


# Generate Unique comments

In [4]:
# Step 2.25: Extract unique comments from all sheets
# This creates a deduplicated list of all comments across the workbook

all_comments = []
comment_sources = []  # Track which sheet each comment came from

if sheets_dict:
    print("\n🔍 Extracting unique comments from all sheets...")
    
    for sheet_name, df in sheets_dict.items():
        # Look for "Comment" column (case-insensitive)
        comment_cols = [col for col in df.columns if col.lower() == 'comment']
        
        if comment_cols:
            comment_col = comment_cols[0]  # Take the first match
            print(f"  Found 'Comment' column in sheet: {sheet_name}")
            
            # Extract comments, remove nulls and empty strings
            comments = df[comment_col].dropna().astype(str)
            comments = comments[comments.str.strip() != '']  # Remove empty/whitespace-only
            
            comment_list = comments.tolist()
            all_comments.extend(comment_list)
            comment_sources.extend([f"{sheet_name}:{comment_col}"] * len(comment_list))
            
            print(f"    - Extracted {len(comment_list)} comments")
        else:
            print(f"  No 'Comment' column found in sheet: {sheet_name}")
    
    if all_comments:
        # Create DataFrame and deduplicate
        comments_df = pd.DataFrame({
            'Comment': all_comments,
            'source': comment_sources
        })
        
        # Track duplicates before removing
        original_count = len(comments_df)
        comments_df = comments_df.drop_duplicates(subset=['Comment'], keep='first')
        unique_count = len(comments_df)
        duplicates_removed = original_count - unique_count
        
        print(f"\n✓ Comment extraction complete:")
        print(f"  - Total comments extracted: {original_count}")
        print(f"  - Duplicates removed: {duplicates_removed}")
        print(f"  - Unique comments: {unique_count}")
        
        # Export unique comments (only the Comment column)
        unique_comments_df = comments_df[['Comment']].copy()
        unique_comments_df.to_csv('../data/processed/unique_comments.csv', index=False)
        print("✓ Unique comments saved to data/processed/unique_comments.csv")
        
        # Show sample
        if unique_count > 0:
            print(f"Sample comment: {unique_comments_df['Comment'].iloc[0][:100]}...")
    else:
        print("✗ No comments found in any sheets with 'Comment' column.")
else:
    print("No workbook loaded. Please check Step 1.")


🔍 Extracting unique comments from all sheets...
  Found 'Comment' column in sheet: COLA FONL Unique
    - Extracted 2517 comments
  Found 'Comment' column in sheet: General Inqury Unique
    - Extracted 1582 comments
  Found 'Comment' column in sheet: Contact Reason Details
    - Extracted 31865 comments

✓ Comment extraction complete:
  - Total comments extracted: 35964
  - Duplicates removed: 11993
  - Unique comments: 23971
✓ Unique comments saved to data/processed/unique_comments.csv
Sample comment:  	ADDRESS VERIFICATION FOR PERMIT FL-I-15562...


In [5]:
# Step 3: Apply BERTopic for theme extraction
model_choice = globals().get('model_choice', 'bertopic')
# Set default sample size for large datasets
sample_size = globals().get('sample_size', 1000)

if all_texts:
    print(f"Running {model_choice.upper()} analysis on {len(all_texts)} texts...")
    
    # Handle large datasets by sampling
    if len(all_texts) > 50000:
        print(f"Dataset is large ({len(all_texts)} texts). Sampling to {sample_size} for analysis to avoid memory issues.")
        sample_indices = np.random.choice(len(all_texts), sample_size, replace=False)
        analysis_texts = [all_texts[i] for i in sample_indices]
        analysis_sources = [text_sources[i] for i in sample_indices]
    else:
        analysis_texts = all_texts
        analysis_sources = text_sources

    # Import additional libraries for custom models
    from umap import UMAP
    from hdbscan import HDBSCAN
    
    # Initialize BERTopic model with optimized settings for large datasets
    topic_model = BERTopic(
        language="english",
        # calculate_probabilities=False,  # Set to False to reduce memory usage
        verbose=True,
        min_topic_size=10,  # Increased from 5 to reduce number of topics and memory
        nr_topics="auto",   # Auto-determine number of topics
        low_memory=True,  # Enable low memory mode for large datasets
        umap_model=UMAP(n_jobs=1),  # Disable multiprocessing in UMAP
        hdbscan_model=HDBSCAN(core_dist_n_jobs=1)  # Disable multiprocessing in HDBSCAN
    )
    
    # Fit the model
    topics, probabilities = topic_model.fit_transform(analysis_texts)
    
    # Get topic information
    theme_info = topic_model.get_topic_info()
    
    print(f"\n✓ BERTopic identified {len(theme_info)-1} topics (excluding outliers)")
    
    # Display extracted themes
    display(theme_info.head(len(theme_info)-1))
    
else:
    print("No texts available for analysis. Please check Step 2.")
    theme_info = None

2026-01-31 14:11:16,102 - BERTopic - Embedding - Transforming documents to embeddings.


Running BERTOPIC analysis on 107922 texts...
Dataset is large (107922 texts). Sampling to 1000 for analysis to avoid memory issues.


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2026-01-31 14:11:18,780 - BERTopic - Embedding - Completed ✓
2026-01-31 14:11:18,780 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-31 14:11:24,213 - BERTopic - Dimensionality - Completed ✓
2026-01-31 14:11:24,214 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-31 14:11:24,221 - BERTopic - Cluster - Completed ✓
2026-01-31 14:11:24,221 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2026-01-31 14:11:24,230 - BERTopic - Representation - Completed ✓
2026-01-31 14:11:24,231 - BERTopic - Topic reduction - Reducing number of topics
2026-01-31 14:11:24,234 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-31 14:11:24,239 - BERTopic - Representation - Completed ✓
2026-01-31 14:11:24,239 - BERTopic - Topic reduction - Reduced number of topics from 40 to 17



✓ BERTopic identified 16 topics (excluding outliers)


,Topic,Count,Name,Representation,Representative_Docs
0,-1,76,-1_label_to_explained_applicant,"[label, to, explained, applicant, on, the, im,...",[IM shared information regarding the needs cor...
1,0,332,0_issues_labeling_colafonl_questions,"[issues, labeling, colafonl, questions, formul...","[Labeling Questions, Labeling Questions, Label..."
2,1,104,1_wine_ct_x000d_he,"[wine, ct, x000d, he, is, red, puerto, percent...","[Wine, Wine, Wine]"
3,2,74,2_correction_clarification_needs_ds,"[correction, clarification, needs, ds, appeale...","[Needs Correction- Clarification, Needs Correc..."
4,3,56,3_transfer_calls_extensions_divisions,"[transfer, calls, extensions, divisions, call,...",[Transfer calls to other Extensions or Divisio...
5,4,54,4_spirits_distilled__,"[spirits, distilled, , , , , , , , ]","[Distilled Spirits, Distilled Spirits, Distill..."
6,5,54,5_new_registration_permit_user,"[new, registration, permit, user, nrc, iris, a...","[New User Registration, New User Registration,..."
7,6,38,6_general_inquiry_answered_sulfte,"[general, inquiry, answered, sulfte, answer, n...","[General Inquiry, General Inquiry, General Inq..."
8,7,37,7____,"[, , , , , , , , , ]","[-, -, -]"
9,8,36,8_other___,"[other, , , , , , , , , ]","[Other, Other, Other]"


In [ ]:
# Step 4: Visualize topics
# Safely get variables that may not exist in the current kernel/session
# theme_info = globals().get('theme_info', None)
# model_choice = globals().get('model_choice', 'bertopic')
# topic_model = globals().get('topic_model', None)
# analysis_texts = globals().get('analysis_texts', [])

from IPython.display import display, HTML
import plotly.io as pio

# CRITICAL: Use 'json' renderer to prevent auto-display blocking
# This allows manual HTML embedding via _display_fig() without hanging
try:
    pio.renderers.default = 'json'
except Exception:
    pass

def _display_fig(fig, title=None):
    """Embed a Plotly figure as HTML to avoid renderer blocking issues."""
    try:
        html = fig.to_html(full_html=False, include_plotlyjs='cdn')
        if title:
            display(HTML(f"<h4>{title}</h4>"))
        display(HTML(html))
    except Exception as e:
        print(f"Failed to render figure as HTML: {e}")

if theme_info is not None and topic_model is not None:
    print("Generating visualization...")
    print(f"Dataset size: {len(analysis_texts)} texts, {len(theme_info)} topics")
    
    # Topic distribution bar chart - fast and informative
    try:
        print("\nGenerating topic barchart...")
        fig_barchart = topic_model.visualize_barchart(top_n_topics=15)
        _display_fig(fig_barchart, title='Top 15 Topics by Frequency')
        print("\n✓ Visualization complete!")
    except Exception as e:
        print(f"Barchart visualization failed: {e}")
    
else:
    if theme_info is None:
        print("No themes available for visualization. Run the analysis cells first.")
    elif model_choice == 'bertopic' and topic_model is None:
        print("BERTopic model object not found in the session. Re-run the BERTopic analysis cell.")
    else:
        print("No visualizations available for the current configuration.")


In [ ]:
# Step 5: Export themes for validation/import into QDA tool
if theme_info is not None:
    # Export to CSV
    output_file = f"../data/results/extracted_themes_{model_choice}.csv"
    theme_info.to_csv(output_file, index=False)
    print(f"✓ Themes exported to '{output_file}'")
    
    # Export detailed topic information
    # Get topic representations
    topic_representations = {}
    for topic_id in theme_info['Topic'].unique():
        if topic_id != -1:  # Skip outlier topic
            words = topic_model.get_topic(topic_id)
            topic_representations[topic_id] = words
    
    # Create detailed export
    detailed_themes = []
    for topic_id, words in topic_representations.items():
        detailed_themes.append({
            'topic_id': topic_id,
            'topic_name': f"Topic_{topic_id}",
            'top_words': ', '.join([word for word, _ in words[:10]]),
            'word_scores': ', '.join([f"{word}:{score:.3f}" for word, score in words[:10]]),
            'document_count': theme_info[theme_info['Topic'] == topic_id]['Count'].iloc[0]
        })
    
    detailed_df = pd.DataFrame(detailed_themes)
    detailed_output = f"../data/results/detailed_themes_{model_choice}.csv"
    detailed_df.to_csv(detailed_output, index=False)
    print(f"✓ Detailed themes exported to '{detailed_output}'")
    
    # Export text-topic assignments for QDA validation
    text_topics_df = pd.DataFrame({
        'text': analysis_texts,
        'source': analysis_sources,
        'topic': topics,
        'topic_probability': [None] * len(topics)  # Probabilities disabled for memory efficiency
    })

    
    assignments_file = f"../data/results/text_topic_assignments_{model_choice}.csv"
    text_topics_df.to_csv(assignments_file, index=False)
    print(f"✓ Text-topic assignments exported to '{assignments_file}'")
    
    print("\n" + "="*50)
    print("EXPORT COMPLETE")
    print("="*50)
    print("Files ready for import into QDA tools like:")
    print("- Taguette (import CSV files)")
    print("- QualCoder (import CSV files)")
    print("- NVivo (import CSV files)")
    print("- ATLAS.ti (import CSV files)")
    
else:
    print("No themes to export.")

## Next Steps

1. **Validate AI-generated themes** in your QDA tool
2. **Refine topics** by merging, splitting, or renaming as needed
3. **Code additional texts** using the validated themes
4. **Export coded data** for further quantitative analysis

## Tips for Better Results

- **Preprocessing**: Clean your text data before analysis (remove noise, standardize formats)
- **Model Tuning**: Adjust `min_topic_size` in BERTopic for more/fewer topics
- **Language**: Set the correct language if your texts aren't in English
- **Scale**: For large datasets (>10k texts), consider running on Databricks Community Edition

## Troubleshooting

- **Memory issues**: Reduce `min_topic_size` or use Top2Vec instead
- **Poor topics**: Preprocess text better or try different model settings
- **No texts found**: Check column names in your Excel file